https://www.analyticsvidhya.com/blog/2019/03/learn-to-use-elmo-to-extract-features-from-text/

#### to load elmo to local disk:
#download the model to local so it can be used again and again
$ mkdir module/module_elmo2
# Download the module, and uncompress it to the destination folder. 
$ curl -L "https://tfhub.dev/google/elmo/2?tf-hub-format=compressed" | tar -zxvC module/module_elmo2

In [ ]:
#data science
import pandas as pd
import numpy as np
import sklearn as skl
from sklearn.metrics.pairwise import cosine_similarity
import random
from matplotlib import pyplot as plt

#NLP
#import spacy
import re

import time
from tqdm import tqdm  # progress bar

# file loading
import pickle

#deep learning
import tensorflow as tf
import tensorflow_hub as hub


In [ ]:
# It's a Tesla V100 with a P3, K80 with P2 
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
pd.set_option('display.max_colwidth', 200)

In [ ]:
# resumes only for now
# gives df with list of strings (tokenized) as well as lemmatized list of strings

ec2_resumes_path = '/home/ubuntu/NLP_projects/job_recommender_project/data/large_files/lf_cleaned_lemmatized_tokenized_resumes.csv'
ec2_pickle_resumes_path = '/home/ec2-user/NLP_projects/job_recommender_project/data/resumes_with_list_of_list.pickle'

resumes = pd.read_pickle(ec2_pickle_resumes_path)


In [ ]:
resumes.head()

In [ ]:
type(resumes['lol'][0][0][0])

In [ ]:
resumes['los'] = resumes['lol']

In [ ]:
len(resumes['los'][0])

In [ ]:
for i in range (0, len(resumes['los'])):
    for j in range (0, len(resumes['los'][i])):
        resumes['los'][i][j] = ' '.join(resumes['los'][i][j])

In [ ]:
resumes['los'][0][3]

In [ ]:
elmo_in = resumes['los']


In [ ]:
# # the lenghts of each resume and resume ID for reference

# for x in range(len(elmo_in)):
#     print(x, ':  ', len(elmo_in[x]))

#     #elmo_in[28] has 212 sentences
#     #
#     #elmo_in[34] has 945 sentences
#     #elmo_in[75] has 3 sentences

In [ ]:
# let's examine resume length in sentences


lengths = []
for i in range(0, len(elmo_in)):
    lengths.extend([len(elmo_in[i])])

#lengths

In [ ]:
# let's look at distributions of resume length

plt.hist(lengths, bins=range(0, 1000, 10))

plt.show()


In [ ]:
plt.hist(lengths, bins=range(0, 200, 5))

plt.show()

In [ ]:
print("example of a short resume:")
print("resume number 244")
print("resume length: {} sentences".format(len(elmo_in[244])))
print(elmo_in[244])

In [ ]:
print("example of a medium resume:")
print("resume number 202")
print("resume length: {} sentences".format(len(elmo_in[202])))
print(elmo_in[202])

In [ ]:
print("example of a long resume:")
print("resume number 28")
print("resume length: {} sentences".format(len(elmo_in[28])))
print(elmo_in[28])

In [ ]:
print('min: {}'.format(np.amin(lengths)))
print('max: {}'.format(np.amax(lengths)))
print('median: {}'.format(np.median(lengths)))
print('mean: {}'.format(np.mean(lengths)))
print('stdev: {}'.format(np.std(lengths)))


In [ ]:
lengths_copy = lengths.copy()


In [ ]:
elmo_in_smalls = elmo_in.copy()

In [ ]:
elmo_in_smalls = elmo_in_smalls.tolist()

In [ ]:
type(elmo_in_smalls)

In [ ]:
for i in elmo_in_smalls:
    if len(i) > 100:
        elmo_in_smalls.remove(i)
    if len(i) <2:
        elmo_in_smalls.remove(i)

#elmo_in_smalls

In [ ]:
print('[0] ',elmo_in_smalls[0])

In [ ]:
print('[0][3] ', type(elmo_in_smalls[0][3]), len(elmo_in_smalls[0][3]), elmo_in_smalls[0][3])

In [ ]:
for i in range(0, len(elmo_in_smalls)):
    for j in range (0, len(elmo_in_smalls[i])):
        if len(elmo_in_smalls[i][j])<2:
            print(i, j, elmo_in_smalls[i][j])



In [ ]:
len(elmo_in_smalls)

In [ ]:
lengths_smalls = []
for i in range(0, len(elmo_in_smalls)):
    lengths_smalls.extend([len(elmo_in_smalls[i])])

#lengths_smalls

In [ ]:
# let's look at distributions of resume length

plt.hist(lengths_smalls, bins=range(0, 100, 5))

plt.show()


In [ ]:
# now we have lengths_smalls for each length
# now we have elmo_in_smalls for each resume

In [ ]:
type(elmo_in_smalls[0])

In [ ]:
# I have elmo saved locally
elmo = hub.Module("/home/ec2-user/module/module_elmo2", trainable=False)



In [ ]:
len(elmo_in_smalls)

In [ ]:
#### SAVING ELMO_IN_SMALLS
type(elmo_in_smalls)
# save elmo in smalls
np_elmo_in_smalls = np.asarray(elmo_in_smalls)
np.save('elmo_resumes_under_100_sentences', np_elmo_in_smalls) 

In [ ]:
# first_try = []
# for i in range(0, len(elmo_in_smalls[0])):
#     first_try = elmo_vectors(elmo_in_smalls[0][i])
#     print(first_try)


#trying on elmo_in_smalls[0]
embeddings = elmo(elmo_in_smalls[0], signature="default",as_dict=True)["elmo"]
print("embeddings is type {} and shape {}".format(type(embeddings), embeddings.shape))
with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    message_embeddings_2d = session.run(tf.reduce_mean(embeddings,axis=0))
    print("message_embeddings_2d is type {} and shape {}".format(type(message_embeddings_2d), message_embeddings_2d.shape))
    message_embeddings_1d = tf.reduce_mean(tf.convert_to_tensor(message_embeddings_2d), axis = 0, keepdims=True)
    print("message_embeddings_1d is type {} and shape {}".format(type(message_embeddings_1d), message_embeddings_1d.shape))


In [ ]:
embeddings[0][0][0]

In [ ]:
embeddings = tf.dtypes.cast(embeddings, tf.float16)  # [1, 2], dtype=tf.int32

In [ ]:
embeddings[0][0][0]

In [ ]:
with tf.Session() as sess:  print(message_embeddings_1d.eval()) 

In [ ]:
#trying on elmo_in_smalls[1]
embeddings1 = elmo(elmo_in_smalls[1], signature="default",as_dict=True)["elmo"]
print("embeddings is type {} and shape {}".format(type(embeddings1), embeddings1.shape))
with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    message_embeddings1_2d = session.run(tf.reduce_mean(embeddings1,axis=0))
    print("message_embeddings1_2d is type {} and shape {}".format(type(message_embeddings1_2d), message_embeddings1_2d.shape))
    message_embeddings1_1d = session.run(tf.reduce_mean(tf.convert_to_tensor(message_embeddings1_2d), axis = 0, keepdims=True))
    print("message_embeddings1_1d is type {} and shape {}".format(type(message_embeddings1_1d), message_embeddings1_1d.shape))

    

In [ ]:
print(message_embeddings1_1d)

In [ ]:
with tf.Session() as sess:  print(message_embeddings1_1d.eval()) 

In [ ]:
with tf.Session() as sess: print(cosine_similarity(message_embeddings_1d.eval(), message_embeddings1_1d.eval()))


In [ ]:
# print("embeddings shape: {}".format(embeddings.shape))
# print("elmo_in_smalls[0] sentences: {}".format(len(elmo_in_smalls[0])))
# max_sentence_length = 0
# for i in range(0, len(elmo_in_smalls[0])):
#     if len(elmo_in_smalls[0][i].split()) > max_sentence_length:
#         max_sentence_length = len(elmo_in_smalls[0][i])
                                  
# print("elmo_in_smalls[0] max sentence length: {}".format(max_sentence_length))


In [ ]:
len(elmo_in_smalls)

In [ ]:
len(elmo_in_smalls[129])

In [ ]:
### try this right after imports to load elmo_in_smalls
elmo_in_smalls = np.load('/home/ec2-user/NLP_projects/job_recommender_project/elmo_resumes_under_100_sentences.npy', allow_pickle=True).tolist()


Many attempts
============

In [ ]:
#data science
import pandas as pd
import numpy as np
import sklearn as skl
from sklearn.metrics.pairwise import cosine_similarity
import random
from matplotlib import pyplot as plt

#NLP
#import spacy
import re

import time
from tqdm import tqdm  # progress bar

# file loading
import pickle

#deep learning
import tensorflow as tf
import tensorflow_hub as hub

from time import process_time

### try this right after imports to load elmo_in_smalls
elmo_in_smalls = np.load('/home/ec2-user/NLP_projects/job_recommender_project/elmo_resumes_under_100_sentences.npy', allow_pickle=True).tolist()


## this was the instantiation of elmo
elmo = hub.Module("/home/ec2-user/module/module_elmo2", trainable=False)

# for all embeddings
#resume_embeddings_append_regular = []
resume_embeddings_append_transpose = []
#resume_embeddings_extend_regular = []
#resume_embeddings_extend_transpose = []




for i in tqdm(range(0, len(elmo_in_smalls))):
    t1 = process_time()
    # save np array of embeddings
    if i % 50 == 0:
        works_ELMo_embeddings_resumes = np.asarray(resume_embeddings_append_transpose)
        np.save('ELMo_embeddings_resumes_0to_{}'.format(i), works_ELMo_embeddings_resumes) 
    
    #print('elmo_in_smalls[{}]'.format(i))
    embeddings_3d = elmo(elmo_in_smalls[i], signature="default",as_dict=True)["elmo"]
    #print('3d size: {}'.format(embeddings_3d.shape))
    #print('3d type ', type(embeddings_3d))
    t2 = process_time()
    
    with tf.Session() as session:
        session.run([tf.global_variables_initializer(), tf.tables_initializer()])
        embeddings_2d = session.run(tf.reduce_mean(embeddings_3d,axis=0))
        #print('2d size: {}'.format(embeddings_2d.shape))
        #print('2d type ', type(embeddings_2d))
        t3 = process_time()
        
        
        embeddings_1d = session.run(tf.reduce_mean(tf.convert_to_tensor(embeddings_2d), axis = 0, keepdims=True))
        #print('1d size: {}'.format(embeddings_1d.shape))
        #print('1d type ', type(embeddings_1d))
        t4 = process_time()
        
        transpose = embeddings_1d.T
        #print(transpose.shape)
        #print('transpose type: ', type(transpose))
        t5 = process_time()
        
        resume_embeddings_append_regular.append(embeddings_1d)
        t6 = process_time()
        
        resume_embeddings_append_transpose.append(transpose)
        t7 = process_time()
        
        resume_embeddings_extend_regular.extend(embeddings_1d)
        t8 = process_time()
        
        resume_embeddings_extend_transpose.extend(transpose)
        t9 = process_time()
        
        #print('length of resume_embeddings_append_regular: {}'.format(len(resume_embeddings_append_regular)))
        #print('length of resume_embeddings_append_transpose: {}'.format(len(resume_embeddings_append_transpose)))
        #print('length of resume_embeddings_extend_regular: {}'.format(len(resume_embeddings_extend_regular)))
        #print('length of resume_embeddings_extend_transpose: {}'.format(len(resume_embeddings_extend_transpose)))      
        
        #print('shape of resume_embeddings_append_regular[{}]: {}'.format(i, resume_embeddings_append_regular[i].shape))
        #print('shape of resume_embeddings_append_transpose[{}]: {}'.format(i, resume_embeddings_append_transpose[i].shape))                                                                        
        #print('shape of resume_embeddings_extend_regular[{}]: {}'.format(i, resume_embeddings_extend_regular[i].shape))                                                                        
        #print('shape of resume_embeddings_extend_transpose[{}]: {}'.format(i, resume_embeddings_extend_transpose[i].shape))                                                                        
               
    
    
#         print('Run {}'.format(i))
#         print('all times from start of run {}'.format(i))
#         print('3d embedding time: {}'.format(t2 - t1))
#         print('2d embedding time: {}'.format(t3 - t2))
#         print('1d embedding time: {}'.format(t4 - t3))
#         print('time to transpose (t5-t4): {}'.format(t5 - t4))
#         print('append_regular time (t6-t5): {}'.format(t6-t5))
#         print('append_transpose time (t7-t6): {}'.format(t7-t6))
#         print('extend_regular time (t8-t7): {}'.format(t8-t7))
#         print('extend_transpose time (t9-t8): {}'.format(t9-t8))
#         print('Total time run {}: {}'.format(i, t9-t1))         
#         print('\n\n\n\t\t\tEND OF RUN')
    
    
    
    
    

In [ ]:
WARNING:tensorflow:From train_elmo.py:31: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.

WARNING:tensorflow:From train_elmo.py:46: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

2020-04-23 16:40:21.396830: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2020-04-23 16:40:21.417314: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300070000 Hz
2020-04-23 16:40:21.417704: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x4b1b840 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-04-23 16:40:21.417730: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
WARNING:tensorflow:From train_elmo.py:51: The name tf.global_variables_initializer is deprecated. Please use tf.compat.v1.global_variables_initializer instead.

WARNING:tensorflow:From train_elmo.py:51: The name tf.tables_initializer is deprecated. Please use tf.compat.v1.tables_initializer instead.

<br>
<br>
<br><br>
<br>
<br>
<br>
<br>
<br>
======================================================
<br>
<br>
<br>
<br>
<br>
<br>
<br>


Moving tf.sess out of for loop
=======================

In [2]:
# import numpy as np
# elmo_in_smalls = np.load('/home/ec2-user/NLP_projects/job_recommender_project/elmo_resumes_under_100_sentences.npy', allow_pickle=True).tolist()


In [14]:
# np.save('/home/ec2-user/NLP_projects/job_recommender_project/elmo_resumes_under_100_sentences', np.asarray(elmo_in_smalls))

In [15]:
for i in range(0, len(elmo_in_smalls)):
    print(i, len(elmo_in_smalls[i]))

0 35
1 9
2 65
3 63
4 46
5 35
6 56
7 76
8 50
9 4
10 15
11 32
12 60
13 33
14 87
15 65
16 26
17 7
18 31
19 28
20 31
21 14
22 36
23 4
24 24
25 72
26 15
27 15
28 31
29 69
30 50
31 41
32 29
33 50
34 6
35 22
36 15
37 12
38 64
39 11
40 22
41 6
42 12
43 25
44 6
45 40
46 8
47 61
48 10
49 17
50 17
51 14
52 32
53 11
54 17
55 9
56 26
57 57
58 20
59 32
60 3
61 15
62 3
63 21
64 28
65 23
66 70
67 31
68 30
69 17
70 92
71 9
72 3
73 74
74 32
75 2
76 68
77 3
78 30
79 23
80 26
81 3
82 18
83 40
84 35
85 61
86 2
87 20
88 93
89 26
90 42
91 47
92 42
93 24
94 100
95 42
96 45
97 25
98 12
99 26
100 88
101 5
102 30
103 46
104 60
105 57
106 36
107 7
108 58
109 35
110 27
111 35
112 16
113 58
114 37
115 32
116 19
117 26
118 4
119 33
120 2
121 11
122 74
123 26
124 51
125 4
126 41
127 19
128 66
129 3
130 40
131 68
132 17
133 55
134 35
135 54
136 20
137 27
138 3
139 8
140 59
141 16
142 35
143 36
144 27
145 35
146 30
147 7
148 47
149 12
150 12
151 2
152 40
153 45
154 30
155 55
156 25
157 3
158 64
159 71
160 4
161 52
162 

In [ ]:
#data science
import pandas as pd
import numpy as np
import sklearn as skl
from sklearn.metrics.pairwise import cosine_similarity
import random
from matplotlib import pyplot as plt

#NLP
#import spacy
import re

import time
from tqdm import tqdm  # progress bar

# file loading
import pickle

#deep learning
import tensorflow as tf
import tensorflow_hub as hub

from time import process_time
import resource

### try this right after imports to load elmo_in_smalls
elmo_in_smalls = np.load('/home/ec2-user/NLP_projects/job_recommender_project/elmo_resumes_under_100_sentences.npy', allow_pickle=True).tolist()


##################reset default graph tf###################
tf.reset_default_graph()
############################################

## this was the instantiation of elmo
elmo = hub.Module("/home/ec2-user/module/module_elmo2", trainable=False)

# for all embeddings
resume_embeddings_append_regular = []
resume_embeddings_append_transpose = []
resume_embeddings_extend_regular = []
resume_embeddings_extend_transpose = []


print('\n\nstarting session\n\n')
t0 = process_time()
with tf.Session() as session:
    t00 = process_time()
    print('\nstarting tf.Session took: {}'.format(t00-t0))
    
    t_pre_initialize_vars = process_time()
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    t_post_initialize_vars = process_time()
    print('Time to initialize variables: {}'.format(t_post_initialize_vars-t_pre_initialize_vars))

    
    
    print('\n\nstarting for loop\n\n')
    for i in tqdm(range(0, 49)):#len(elmo_in_smalls))):
        t1 = process_time()
        # save np array of embeddings
        if i % 50 == 0:
            np_ELMo_embeddings_resumes = np.asarray(resume_embeddings_append_transpose)
            np.save('new_ELMo_embeddings_resumes_0_to_{}'.format(i), np_ELMo_embeddings_resumes) 

        print('elmo_in_smalls[{}]'.format(i))
        embeddings_3d = elmo(elmo_in_smalls[i], signature="default",as_dict=True)["elmo"]
        embeddings_3d_f16 = tf.dtypes.cast(embeddings_3d, tf.float16)
        print('3d size: {}'.format(embeddings_3d_f16.shape))
        print('3d type ', type(embeddings_3d_f16))
        t2 = process_time()

        
        embeddings_2d = session.run(tf.reduce_mean(embeddings_3d_f16,axis=0))
        print('2d size: {}'.format(embeddings_2d.shape))
        print('2d type ', type(embeddings_2d))
        t3 = process_time()


        embeddings_1d = session.run(tf.reduce_mean(tf.convert_to_tensor(embeddings_2d), axis = 0, keepdims=True))
        print('1d size: {}'.format(embeddings_1d.shape))
        print('1d type ', type(embeddings_1d))
        t4 = process_time()

        transpose = embeddings_1d.T
        print(transpose.shape)
        print('transpose type: ', type(transpose))
        t5 = process_time()

        resume_embeddings_append_regular.append(embeddings_1d)
        t6 = process_time()

        resume_embeddings_append_transpose.append(transpose)
        t7 = process_time()

        resume_embeddings_extend_regular.extend(embeddings_1d)
        t8 = process_time()

        resume_embeddings_extend_transpose.extend(transpose)
        t9 = process_time()

        print('length of resume_embeddings_append_regular: {}'.format(len(resume_embeddings_append_regular)))
        print('length of resume_embeddings_append_transpose: {}'.format(len(resume_embeddings_append_transpose)))
        print('length of resume_embeddings_extend_regular: {}'.format(len(resume_embeddings_extend_regular)))
        print('length of resume_embeddings_extend_transpose: {}'.format(len(resume_embeddings_extend_transpose)))      

        print('shape of resume_embeddings_append_regular[{}]: {}'.format(i, resume_embeddings_append_regular[i].shape))
        print('shape of resume_embeddings_append_transpose[{}]: {}'.format(i, resume_embeddings_append_transpose[i].shape))                                                                        
        print('shape of resume_embeddings_extend_regular[{}]: {}'.format(i, resume_embeddings_extend_regular[i].shape))                                                                        
        print('shape of resume_embeddings_extend_transpose[{}]: {}'.format(i, resume_embeddings_extend_transpose[i].shape))                                                                        

        #not .extend?



        print('\nRun {}'.format(i))
        
        print('Iteration ', i, ' maxrss: ', resource.getrusage(resource.RUSAGE_SELF).ru_maxrss)
        print('graph size: {}'.format(tf.graph.shape))
        print('\nall times from start of run {}\n'.format(i))
        print('3d embedding time: {}'.format(t2 - t1))
        print('2d embedding time: {}'.format(t3 - t2))
        print('1d embedding time: {}'.format(t4 - t3))
        print('time to transpose (t5-t4): {}'.format(t5 - t4))
        print('append_regular time (t6-t5): {}'.format(t6-t5))
        print('append_transpose time (t7-t6): {}'.format(t7-t6))
        print('extend_regular time (t8-t7): {}'.format(t8-t7))
        print('extend_transpose time (t9-t8): {}'.format(t9-t8))
        print('Total time run {}: {}'.format(i, t9-t1))         
        print('\n\n\n\t\t\tEND OF RUN')

    
    
    
    

Expanding graph problem?? *******TRY ME********
=====================


- maybe the answer is to elmo_3d every resume...then do something?

In [18]:
###### data science
import pandas as pd
import numpy as np
import sklearn as skl
from sklearn.metrics.pairwise import cosine_similarity
import random
from matplotlib import pyplot as plt

#NLP
#import spacy
import re

import time
from tqdm import tqdm  # progress bar

# file loading
import pickle

#deep learning
import tensorflow as tf
import tensorflow_hub as hub

from time import process_time
import resource

### try this right after imports to load elmo_in_smalls
elmo_in_smalls = np.load('/home/ec2-user/NLP_projects/job_recommender_project/elmo_resumes_under_100_sentences.npy', allow_pickle=True).tolist()
elmo_chec = elmo_in_smalls.copy

##################reset default graph tf###################
tf.reset_default_graph()
############################################

## this was the instantiation of elmo
elmo = hub.Module("/home/ec2-user/module/module_elmo2", trainable=False)

# for all embeddings
# resume_embeddings_append_regular = []
resume_embeddings_append_transpose = []
# resume_embeddings_extend_regular = []
# resume_embeddings_extend_transpose = []


# print('\n\nstarting session\n\n')
t0 = process_time()
with tf.Session() as session:
    t00 = process_time()
#     print('\nstarting tf.Session took: {}'.format(t00-t0))
    
    t_pre_initialize_vars = process_time()
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    t_post_initialize_vars = process_time()
#     print('Time to initialize variables: {}'.format(t_post_initialize_vars-t_pre_initialize_vars))

    
    
#     print('\n\nstarting for loop\n\n')
    for i in tqdm(range(250, 355)):
        t1 = process_time()
        # save np array of embeddings
        if i % 50 == 0:
            np_ELMo_embeddings_resumes = np.asarray(resume_embeddings_append_transpose)
            np.save('ELMo_embeddings_resumes_250_to_{}'.format(i), np_ELMo_embeddings_resumes) 

            
            
            
#         assign_op = x.assign(1)
#         sess.run(assign_op)  # or `assign_op.op.run()`
#         print(x.eval())
    
#         print('elmo_in_smalls[{}]'.format(i))
        embeddings_3d = elmo(elmo_in_smalls[i], signature="default",as_dict=True)["elmo"]
#         print('3d type original', type(embeddings_3d))
        t_make_np_start = process_time()
        embeddings_3d_np = embeddings_3d.eval()
        t_make_np_stop = process_time()          
#         print('3d type np?', type(embeddings_3d_np))
        
        
        
        embeddings_3d_np_f16 = embeddings_3d_np #tf.dtypes.cast(embeddings_3d_np, tf.float16)
#         print('3d size: {}'.format(embeddings_3d_np_f16.shape))
#         print('3d type np? 16?', type(embeddings_3d_np_f16))
        t2 = process_time()

        
        embeddings_2d = np.mean(embeddings_3d_np_f16,axis=0)#session.run(tf.reduce_mean(embeddings_3d_np_f16,axis=0))
#         print('2d size: {}'.format(embeddings_2d.shape))
#         print('2d type ', type(embeddings_2d))
        t3 = process_time()


        embeddings_1d = np.mean(embeddings_2d,axis=0)#session.run(tf.reduce_mean(tf.convert_to_tensor(embeddings_2d), axis = 0, keepdims=True))
#         print('1d size: {}'.format(embeddings_1d.shape))
#         print('1d type ', type(embeddings_1d))
        t4 = process_time()

        transpose = embeddings_1d.T
#         print(transpose.shape)
#         print('transpose type: ', type(transpose))
        t5 = process_time()

#         resume_embeddings_append_regular.append(embeddings_1d)
        t6 = process_time()

        resume_embeddings_append_transpose.append(transpose)
        t7 = process_time()

#         resume_embeddings_extend_regular.extend(embeddings_1d)
        t8 = process_time()

#         resume_embeddings_extend_transpose.extend(transpose)
        t9 = process_time()

#         print('length of resume_embeddings_append_regular: {}'.format(len(resume_embeddings_append_regular)))
#         print('length of resume_embeddings_append_transpose: {}'.format(len(resume_embeddings_append_transpose)))
#         print('length of resume_embeddings_extend_regular: {}'.format(len(resume_embeddings_extend_regular)))
#         print('length of resume_embeddings_extend_transpose: {}'.format(len(resume_embeddings_extend_transpose)))      

#         print('shape of resume_embeddings_append_regular[{}]: {}'.format(i, resume_embeddings_append_regular[i].shape))
#         print('shape of resume_embeddings_append_transpose[{}]: {}'.format(i, resume_embeddings_append_transpose[i].shape))                                                                        
#         print('shape of resume_embeddings_extend_regular[{}]: {}'.format(i, resume_embeddings_extend_regular[i].shape))                                                                        
#         print('shape of resume_embeddings_extend_transpose[{}]: {}'.format(i, resume_embeddings_extend_transpose[i].shape))                                                                        

        #not .extend?



#         print('\nRun {}'.format(i))
        
#         print('Iteration ', i, ' maxrss: ', resource.getrusage(resource.RUSAGE_SELF).ru_maxrss)
        
        print('\nall times from start of run {}\n'.format(i))
        print('3d embedding time: {}'.format(t_make_np_start - t1))
        print('3d tensor to np time: {}'.format(t_make_np_stop - t_make_np_start))
        print('3d make 16b time: {}'.format(t2-t_make_np_stop))
        print('2d embedding time: {}'.format(t3 - t2))
        print('1d embedding time: {}'.format(t4 - t3))
        print('time to transpose (t5-t4): {}'.format(t5 - t4))
#         print('append_regular time (t6-t5): {}'.format(t6-t5))
        print('append_transpose time (t7-t6): {}'.format(t7-t6))
#         print('extend_regular time (t8-t7): {}'.format(t8-t7))
#         print('extend_transpose time (t9-t8): {}'.format(t9-t8))
        print('Total time run {}: {}'.format(i, t9-t1))         
        print('\n\n\n\t\t\tEND OF RUN')

np_ELMo_embeddings_resumes = np.asarray(resume_embeddings_append_transpose)
np.save('ELMo_embeddings_resumes_final', np_ELMo_embeddings_resumes)    
    
    
    

KeyboardInterrupt: 

In [ ]:
resume_embeddings_append_transpose[250]

In [ ]:
## this was the instantiation of elmo
elmo = hub.Module("/home/ec2-user/module/module_elmo2", trainable=False)


# for all embeddings

resume_embeddings = []


for i in tqdm(range(0, 1)):#len(elmo_in_smalls))):
    # save np array of embeddings
    if i % 50 == 0:
        np_ELMo_embeddings_resumes = np.asarray(resume_embeddings)
        np.save('ELMo_embeddings_resumes_401_to_{}'.format(i), np_ELMo_embeddings_resumes) 
    
    print('elmo_in_smalls[{}]'.format(i))
    embeddings_3d = elmo(elmo_in_smalls[i], signature="default",as_dict=True)["elmo"]
    print('3d size: {}'.format(embeddings_3d.shape))
    print('3d type ', type(embeddings_3d))
    
    with tf.Session() as session:
        session.run([tf.global_variables_initializer(), tf.tables_initializer()])
        embeddings_2d = session.run(tf.reduce_mean(embeddings_3d,axis=0))
        print('2d size: {}'.format(embeddings_2d.shape))
        print('2d type ', type(embeddings_2d))
        
        embeddings_1d = session.run(tf.reduce_mean(tf.convert_to_tensor(embeddings_2d), axis = 0, keepdims=True))
        print('1d size: {}'.format(embeddings_1d.shape))
        print('1d type ', type(embeddings_1d))
        
        transpose = embeddings_1d.T
        print(transpose.shape)
        print('transpose type: ', type(transpose))

        resume_embeddings.append(transpose)
        
        print('length of resume_embeddings: {}'.format(len(resume_embeddings)))
        print('shape of resume_embeddings[{}]: {}'.format(i, resume_embeddings[i].shape))
        #not .extend?

        
        #####closing session at everytime
        session.close()
    


    

In [ ]:
len(resume_embeddings_append_regular)

In [ ]:
resume_embeddings_append_regular[0].shape

In [ ]:
resume_embeddings_append_transpose[0]

In [ ]:
f32 = 3/1048
f32

In [ ]:
f16 = np.float16(3/1048)
f16

In [ ]:
resume_embeddings[0].shape

In [ ]:
print(len(resume_embeddings))

In [ ]:
print(type(resume_embeddings))

In [ ]:
type(resume_embeddings[0])

In [ ]:
resume_embeddings[0].shape

In [ ]:
# # save np array of embeddings
# np_ELMo_embeddings_resumes = np.asarray(resume_embeddings)
# np.save('ELMo_embeddings_resumes_401_{}'.format(len(elmo_in_smalls)', np_ELMo_embeddings_resumes) 

In [ ]:
# # save elmo_train_new
# pickle_out = open("ELMo_embeddings_resumes.pickle","wb")
# pickle.dump(np_ELMo_embeddings_resumes, pickle_out)
# pickle_out.close()

# ========================================